<a href="https://colab.research.google.com/github/rdemarqui/topic_modeling/blob/main/customer_claim_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer claims analysis

In [1]:
import warnings

## Open Dataset

The dataset utilized in this work was created from

In [2]:
import pandas as pd

In [3]:
data_url = 'https://raw.githubusercontent.com/rdemarqui/topic_modeling/main/datasets/reclame_aqui_full.csv'
df = pd.read_csv(data_url)
print(df.shape)
df.head()

(500, 5)


,Unnamed: 0.1,Unnamed: 0,Title,link,description
0,0,0,Claro residência,/claro/claro-residencia_h-VgVeL0ez_dO42I/,Estou há mais de 7 dias tentando efetuar a con...
1,1,1,NO CONSIGO FALAR COM UM HUMANO,/claro/no-consigo-falar-com-um-humano_0kiSV_xg...,Realizei o pagamento das faturas atrasadas e s...
2,2,2,Oi virou claro e não recebe ligações!!!,/claro/oi-virou-claro-e-nao-recebe-ligacoes_MG...,Minha mãe tinha um chip da Oi que virou Claro ...
3,3,3,Mudança de plano sem a minha autorização,/claro/mudanca-de-plano-sem-a-minha-autorizaca...,Na sexta-feira dia 6 de outubro fiz uma recarg...
4,4,4,Plano PREZAO,/claro/plano-prezao_iW3NARVJ71tk_jVM/,"Bom diaNa data de 08/10/2023, recebi um SMS in..."


## Text Preprocess

### Normalize

In [4]:
import string

In [5]:
punctuations = list(string.punctuation)

def remove_punctuation(text):
  return ''.join([char if char not in punctuations else ' ' for char in text])

In [6]:
accentuation = {
    "á": "a", "ã": "a", "à": "a","â": "a",
    "é": "e","ê": "e",
    "í": "i",
    "ó": "o","õ": "o", "ô":"o",
    "ú": "u",
    "ç": "c"
    }

def remove_accentuation(text):
  return ''.join(accentuation.get(char, char) for char in text)

In [7]:
special_charaters = [
    "1","2","3","4","5","6","7","8","9","0",
    " a "," b "," c "," d "," e "," f "," g "," h "," i "," j "," k "," l ",
    " m "," n "," o "," p "," q "," r "," s "," t "," u "," v "," x "," z ",
    "r$", "$",
    ]

def remove_special_characters(text):
  return ''.join([char if char not in special_charaters else ' ' for char in text])

In [8]:
text_to_delete = ['editado pelo reclame aqui', 'tim', 'claro', 'vivo']

def remove_text(text):
    for sentence in text_to_delete:
        text = text.replace(sentence, "")
    return text

In [9]:
def normalize_text(df, text_field, lower=True, rem_punct=True, rem_accent=True, rem_spec_caract=True, rem_name=False):
    df[text_field + "_clean"] = df[text_field].astype(str)
    if lower: df[text_field + "_clean"] = df[text_field + "_clean"].str.lower()
    if rem_punct: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_punctuation)
    if rem_accent: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_accentuation)
    if rem_spec_caract: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_special_characters)
    df[text_field + "_clean"] = df[text_field + "_clean"].replace(r'\s+', ' ', regex=True) #remove spaces

In [10]:
normalize_text(df, 'description')

### Stop Words

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
stops_nltk = nltk.corpus.stopwords.words('portuguese')
stop_words = list(stops_nltk)
stop_words = [remove_accentuation(word) for word in stop_words]
stop_words = list(set(stop_words))

def remove_stop_words(text):
    words_list = text.split()
    words_list = [word for word in words_list if word not in stop_words]
    text = ' '.join(words_list)

    return text

In [13]:
df["description_clean_stop"] = df["description_clean"].apply(remove_stop_words)

### Lemmatize

In [14]:
import spacy.cli

language_models = ['pt_core_news_sm', 'pt_core_news_md', 'pt_core_news_lg']
spacy_model = language_models[2]

disable = ['tagger', 'parser', 'ner', 'entity_ruler', 'entity_linker', 'textcat',
           'morphologizer','attribute_ruler']

try:
  nlp = spacy.load(spacy_model, disable=disable)
except:
  spacy.cli.download(spacy_model)
  nlp = spacy.load(spacy_model, disable=disable)

print(nlp.pipe_names)

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
['tok2vec', 'lemmatizer']


In [15]:
def lemma_pipe(doc_list):
  lemma_text_list = []
  for doc in nlp.pipe(doc_list, n_process=-1):
      lemma_text_list.append(" ".join(token.lemma_ for token in doc))

  return lemma_text_list

In [16]:
df["description_clean_stop_lemma"] = lemma_pipe(df["description_clean_stop"])

## Data Analysis

In [17]:
final_docs = list(df['description_clean_stop_lemma'])

## Latent Dirichlet Allocation (LDA)

In [18]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import gensim
import gensim.corpora as corpora

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [19]:
# Tokenize text
token_docs = [word_tokenize(text) for text in final_docs]

# Convert text to frequency dictionary
id2word = corpora.Dictionary(token_docs)

# Convert text in bag of words
corpus = [id2word.doc2bow(text) for text in token_docs]

In [20]:
lda_model = gensim.models.LdaModel(corpus,
                                   num_topics=10,
                                   id2word=id2word,
                                   random_state=42,
                                   update_every=1,
                                   chunksize=50,
                                   passes=15,
                                   alpha="auto")

Imagine the need to find trending topics in Tweets with little pre-processing effort. In this case, one may choose to use Top2Vec and BERTopic. They work splendidly on shorter textual sources and do not require much pre-processing.

Instead, imagine a scenario where a customer is interested in finding how a given document may contain a mixture of multiple topics. In this case, approaches like LDA and NMF would be preferable. BERTopic and Top2Vec assign a document to one topic only. Although the probability distribution of the HDBSCAN may be used as proxy of the topics distribution, BERTopic and Top2Vec are not mixed membership models by design.

## Visualization

In [21]:
try:
  import pyLDAvis.gensim
except:
  !pip install pyLDAvis==2.1.2 -q
  import pyLDAvis.gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [23]:
warnings.filterwarnings("ignore")
vis = pyLDAvis.gensim.prepare(lda_model, corpus, lda_model.id2word, mds="mmds", R=30)
pyLDAvis.display(vis)